# Programming Assignment 4 - Linear versus Ridge Regression 

Your friend Bob just moved to Boston. He is a real estate agent who is trying to evaluate the prices of houses in the Boston area. He has been using a linear regression model but he wonders if he can improve his accuracy on predicting the prices for new houses. He comes to you for help as he knows that you're an expert in machine learning. 

As a pro, you suggest doing a *polynomial transformation*  to create a more flexible model, and performing ridge regression since having so many features compared to data points increases the variance. 

Bob, however, being a skeptic isn't convinced. He wants you to write a program that illustrates the difference in training and test costs for both linear and ridge regression on the same dataset. Being a good friend, you oblige and hence this assignment :) 

In this notebook, you are to explore the effects of ridge regression.  We will use a dataset that is part of the sklearn.dataset package.  Learn more at https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

## Step 1:  Getting, understanding, and preprocessing the dataset

We first import the standard libaries and some libraries that will help us scale the data and perform some "feature engineering" by transforming the data into $\Phi_2({\bf x})$

In [1]:
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold

###  Importing the dataset

In [2]:
# Import the boston dataset from sklearn
boston_data = load_boston()

In [3]:
#  Create X and Y variables - X holding the .data and Y holding .target 
X = boston_data.data
y = boston_data.target

#  Reshape Y to be a rank 2 matrix 
y = y.reshape(X.shape[0], 1)

# Observe the number of features and the number of labels
print('The number of features is: ', X.shape[1])
# Printing out the features
print('The features: ', boston_data.feature_names)
# The number of examples
print('The number of exampels in our dataset: ', X.shape[0])
#Observing the first 2 rows of the data
print(X[0:2])


The number of features is:  13
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
The number of exampels in our dataset:  506
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]


We will also create polynomial feeatures for the dataset to test linear and ridge regression on data with d = 1 and data with d = 2. Feel free to increase the # of degress and see what effect it has on the training and test error. 

In [4]:
# Create a PolynomialFeatures object with degree = 2. 
# Transform X and save it into X_2. Simply copy Y into Y_2 
poly = PolynomialFeatures(degree=2)
X_2 = poly.fit_transform(X)
y_2 = y

In [5]:
# the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
print(X_2.shape)
print(y_2.shape)

(506, 105)
(506, 1)


# Your code goes here

In [6]:
# TODO - Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# TODO - Return w values

def get_coeff_ridge_normaleq(X_train, y_train, alpha):
    # use np.linalg.pinv(a)
    I = np.identity(X_train.shape[1])
    inv = np.dot(X_train.T, X_train) + alpha*I
    inv = np.linalg.pinv(inv)
    w = np.dot(inv, X_train.T)
    w = np.dot(w, y_train)
    return w

In [7]:
# TODO - Define the evaluate_err_ridge function.
# TODO - Return the train_error and test_error values


def evaluate_err(X_train, X_test, y_train, y_test, w):
    y_hat = np.dot(X_train, w)
    ytest_hat = np.dot(X_test, w)
    
    train_error = np.sum(np.square(y_hat - y_train)) / len(X_train)
    test_error = np.sum(np.square(ytest_hat - y_test)) / len(X_test)
    
    return train_error, test_error

In [8]:
# TODO - Finish writting the k_fold_cross_validation function. 
# TODO - Returns the average training error and average test error from the k-fold cross validation
# use Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold_cross_validation(k, X, y, alpha):
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # scaling the data matrix
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Adding a column of 1's to the front of X_train and X_test
        one_col = np.ones((X_train.shape[0],1))
        X_train = np.hstack((one_col, X_train ))
        one_col = np.ones((X_test.shape[0],1))
        X_test  = np.hstack((one_col, X_test ))
        
        # determine the training error and the test error
        w = get_coeff_ridge_normaleq(X_train, y_train, alpha)
        
        train_error, test_error = evaluate_err(X_train, X_test, y_train, y_test, w)
        total_E_val_test += test_error
        total_E_val_train += train_error
        
    E_val_test = total_E_val_test / k
    E_val_train = total_E_val_train / k
    return  E_val_test, E_val_train
    


In [9]:
# For 1(a)
test_val, train_val = k_fold_cross_validation(10, X, y, 0.0)
print("Linear regression model, using 10-fold cross validation.")
print("MSE for test: ", test_val)
print("MSE for train: ", train_val)

Linear regression model, using 10-fold cross validation.
MSE for test:  23.63606860542821
MSE for train:  21.806183575851065


In [10]:
# For 1(b)
alpha = np.logspace(.01, 1, num=13)
sum_train, sum_test = 0, 0
print("Ridge regression model, using 10-fold cross validation.")
for i in alpha:
    test_val, train_val = k_fold_cross_validation(10, X, y, i)
    sum_train += train_val
    sum_test += test_val
    print("When alpha is {}, train val is {}, test val is {}".format(i, train_val, test_val))
   
print("Average MSE for train set is: ", sum_train / len(alpha))
print("Average MSE for test set is: ", sum_test / len(alpha))

Ridge regression model, using 10-fold cross validation.
When alpha is 1.023292992280754, train val is 21.810064209445805, test val is 23.635106720354944
When alpha is 1.2373711899353526, train val is 21.811833657592125, test val is 23.635883832250133
When alpha is 1.4962356560944334, train val is 21.814403182714514, test val is 23.637262110482403
When alpha is 1.8092559102538208, train val is 21.81812978704581, test val is 23.639560342109444
When alpha is 2.1877616239495525, train val is 21.823526530670666, test val is 23.643246542987903
When alpha is 2.6454526947240495, train val is 21.831328668836225, test val is 23.64900296332241
When alpha is 3.198895109691398, train val is 21.842586598001407, test val is 23.657817679646804
When alpha is 3.868120546330522, train val is 21.85879586553974, test val is 23.671112946059374
When alpha is 4.677351412871983, train val is 21.882078198982207, test val is 23.69092414856501
When alpha is 5.6558775708915405, train val is 21.91543252678735, test

In [11]:
# For 1(c)
alpha = np.logspace(1, 5, num=13)
sum_train, sum_test = 0, 0
print("Ridge regression model, using 10-fold cross validation.")
for i in alpha:
    test_val, train_val = k_fold_cross_validation(10, X, y, i)
    sum_train += train_val
    sum_test += test_val
    print("When new alpha is {}, train val is {}, test val is {}".format(i, train_val, test_val))
   
print("Average MSE for train set with new aplha is: ", sum_train / len(alpha))
print("Average MSE for test set with new aplha is: ", sum_test / len(alpha))

Ridge regression model, using 10-fold cross validation.
When new alpha is 10.0, train val is 22.127321102577064, test val is 23.914361582953426
When new alpha is 21.544346900318832, train val is 23.12442428565689, test val is 24.86756740552452
When new alpha is 46.41588833612777, train val is 26.934135882264844, test val is 28.593239993155418
When new alpha is 100.0, train val is 40.18568455612646, test val is 41.70924340688511
When new alpha is 215.44346900318823, train val is 78.51808672478744, test val is 79.87911878294982
When new alpha is 464.15888336127773, train val is 160.252465954425, test val is 161.45030820065497
When new alpha is 1000.0, train val is 278.23915817295574, test val is 279.2368421240491
When new alpha is 2154.4346900318824, train val is 394.5357675343761, test val is 395.27692442129177
When new alpha is 4641.588833612777, train val is 480.5518587815656, test val is 481.0402255237194
When new alpha is 10000.0, train val is 533.7363430617409, test val is 534.0364

In [12]:
# For 2
test_val, train_val = k_fold_cross_validation(10, X_2, y_2, 0.0)
print("Linear regression model, using 10-fold cross validation, with polynomial transformation.")
print("MSE for test: ", test_val)
print("MSE for train: ", train_val)

alpha = np.logspace(.01, 1, num=13)
sum_train, sum_test = 0, 0
print("Ridge regression model, using 10-fold cross validation, with polynomial transformation.")
for i in alpha:
    test_val, train_val = k_fold_cross_validation(10, X_2, y_2, i)
    sum_train += train_val
    sum_test += test_val
    print("When alpha is {}, train val is {}, test val is {}".format(i, train_val, test_val))
   
print("Average MSE for train set is: ", sum_train / len(alpha))
print("Average MSE for test set is: ", sum_test / len(alpha))

Linear regression model, using 10-fold cross validation, with polynomial transformation.
MSE for test:  11.854968234785357
MSE for train:  5.8088208160124655
Ridge regression model, using 10-fold cross validation, with polynomial transformation.
When alpha is 1.023292992280754, train val is 7.4316048993421875, test val is 11.755772615671592
When alpha is 1.2373711899353526, train val is 7.578014628504877, test val is 11.83641981464214
When alpha is 1.4962356560944334, train val is 7.73444342388571, test val is 11.924715611946223
When alpha is 1.8092559102538208, train val is 7.9018145020924475, test val is 12.02130157712696
When alpha is 2.1877616239495525, train val is 8.081235046549736, test val is 12.127135874177302
When alpha is 2.6454526947240495, train val is 8.274115904325258, test val is 12.24368328970293
When alpha is 3.198895109691398, train val is 8.482345997128379, test val is 12.373154458018675
When alpha is 3.868120546330522, train val is 8.708517479370503, test val is 12

In [16]:
# For 3(b)
x = np.array([5, 0.5, 2, 0, 4, 8, 4, 6, 2, 2, 2, 4, 5.5])
x_predict = x.reshape(1, 13)
x2_predict = poly.fit_transform(x_predict)

train_x = X_2
train_y = y_2

train_y_mean = np.mean(train_y)
train_y = train_y - train_y_mean

scaler = preprocessing.StandardScaler().fit(train_x)
train_x = scaler.transform(train_x)
x2_predict = scaler.transform(x2_predict)

w = get_coeff_ridge_normaleq(train_x, train_y, 0)
prediction = np.dot(x2_predict, w) + train_y_mean
print(prediction[0][0])

253.1560947929154
